In [18]:
import json
# import pandas as pd
import os
import requests
import time
import pymysql
import datetime
from fake_useragent import UserAgent
# from tqdm import tqdm
# import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

def conn():
    return pymysql.connect(
        charset='utf8',
        user='root', 
        passwd='', 
        host='localhost',
        db='test'
    )

def headers():
    ua = UserAgent()
    return {'User-Agent' : ua.random}
     

# ----------db도로명주소 > 좌표 insert----------
def addrToCoordinates(result, conn, url, headers):
    cursor = conn.cursor()
    for res in result:
        try:
            link = url + res[0]
            page = requests.get(link, headers=headers)
            time.sleep(1.6)
            json_data = page.json()
        #     print(json_data)
            status = json_data['response']['status']

            if status == 'OK':
                x = json_data['response']['result']['point']['x']
                y = json_data['response']['result']['point']['y']
        #         print('status = {0}\n x = {1} \n y = {2}'.format(status, x, y))
            else :
                print(str(res[1]) + ' no')
                x = 0
                y = 0
        except:
            print('error')
            pass

        try:
            sql = "UPDATE hairshop SET c_lng=%s, c_lat=%s, edit_date=%s WHERE c_idx=%s"
            val = (x, y, datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), res[1])
            cursor.execute(sql, val)
            conn.commit()
        except:
            print("sql-Update Hairshop fail 주소 : {0}".format(res[0]))
    conn.close()
    
# ----------- 단일 주소 변환 테스트 ------------
def changeOne(address, url, headers):
    try:
        link = url + address
        page = requests.get(link, headers=headers)
        time.sleep(1.6)
        json_data = page.json()
        print(json_data)
        status = json_data['response']['status']

        if status == 'OK':
            x = json_data['response']['result']['point']['x']
            y = json_data['response']['result']['point']['y']
            print('status = {0}\n x = {1} \n y = {2}'.format(status, x, y))
        else :
            print('no')
            x = 0
            y = 0
    except:
        print('error')
        pass

# ---------------------------addr, addr_detail 분할--------------------------
def splitAddr(result, conn):
    cursor = conn.cursor()
    for res in result:
        try:
            arrAddr = res[0].split()
            lenAddr = len(arrAddr)
        #     print(arrAddr)
            c_addr = (' ').join(arrAddr[:5])
            c_addr_detail = (' ').join(arrAddr[5:])
        #     print(c_addr)
        #     print(c_addr_detail)
        #     (' ').join(res[0].split())
            try:
                sql = "UPDATE hairshop SET c_addr=%s, c_addr_detail=%s, edit_date=%s WHERE c_idx=%s"
                val = (c_addr, c_addr_detail, datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), res[1])
                cursor.execute(sql, val)
                conn.commit()
            except:
                print("sql-Update Hairshop fail 주소 : {0}".format(c_addr))

        except:
            print("nono - ".foramt(res[1]))
            pass

    conn.close()

In [12]:
headers = headers()
# conn = conn()
# cursor = conn.cursor()

# sql = "SELECT `c_addr`, `c_idx` FROM `hairshop` WHERE `c_idx` > 65332;"
# sql = "SELECT * FROM `hairshop` order by c_visitor_review DESC limit 1000;"
# sql = "SELECT * FROM `hairshop` WHERE `c_search_location` = '창원시청' AND `c_idx` = 670;"
# cursor.execute(sql)
# result = cursor.fetchall()

param1 = 'service=address'
param2 = 'request=getCoord'
param3 = 'key=4A1CF381-0E54-33E4-AC30-0357FFAE3951'
param4 = 'type=road' #PARCEL-지번
param5 = 'refine=true'
url = 'http://api.vworld.kr/req/address?{0}&{1}&{2}&{3}&{4}&address='.format(param1,param2,param3,param4,param5)

changeOne('경남 창원시 의창구 용호동 73-58 2층', url, headers)

# os.system('shutdown -s -f')

{'response': {'service': {'name': 'address', 'version': '2.0', 'operation': 'getCoord', 'time': '27(ms)'}, 'status': 'NOT_FOUND', 'record': {'total': '0', 'current': '0'}, 'page': {'total': '1', 'current': '1', 'size': '10'}}}
no


In [19]:
conn = conn()
cursor = conn.cursor()

# sql = "SELECT c_addr, c_addr_detail, c_idx FROM `hairshop` where c_lat=0 and `c_search_location` = '창원시청';"
sql = "SELECT c_addr, c_addr_detail, c_idx FROM `hairshop2` where `c_search_location` = '창원시청' and c_addr_detail != ''";
# sql = "SELECT * FROM `hairshop` order by c_visitor_review DESC limit 1000;"
# sql = "SELECT * FROM `hairshop` WHERE `c_search_location` = '창원시청' AND `c_idx` = 670;"
cursor.execute(sql)
result = cursor.fetchall()
for res in result:
    val = res[0] + ' ' + res[1]

    try:
        sql = "UPDATE hairshop2 SET c_addr=%s, c_addr_detail=%s, edit_date=%s WHERE c_idx=%s"
        val = (val, '', datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), res[2])
        cursor.execute(sql, val)
        conn.commit()
    except:
        print("sql-Update Hairshop fail 주소 : {0}".format(c_addr))



In [ ]:
import requests
import time
import pymysql
import re
import datetime
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from fake_useragent import UserAgent

ua = UserAgent()
# ua = UserAgent(verify_ssl=False)
headers = {'User-Agent' : ua.random}

conn = pymysql.connect(
    charset='utf8',
    user='root', 
    passwd='', 
    host='localhost',
    db='test'
)
cursor = conn.cursor()

sql = "SELECT c_idx, c_idx_naver FROM `hairshop` WHERE c_lat=0 and c_search_location ='창원시청' and c_idx >242;"
# sql = "SELECT * FROM `hairshop` order by c_visitor_review DESC limit 1000;"
# sql = "SELECT * FROM `hairshop` WHERE `c_search_location` = '창원시청' AND `c_idx` = 670;"
cursor.execute(sql)
result = cursor.fetchall()
# print(result[0][0]) #c_idx
# print(result[0][1]) #c_idx_naver
# print(result[0][2]) #c_name
# print(result[0][3]) #c_search_location
# print(len(result))

for res in result:
    c_home = []

    #----------------------------home---------------------------
    url = 'https://m.place.naver.com/hairshop/{0}/home?entry=pll'.format(res[1])
#     response = requests.get(url, headers=headers)
    driver = webdriver.Chrome("./chromedriver")
    driver.get(url)
    time.sleep(3)
    
    try:
        more = driver.find_element_by_css_selector("a._1Gmk4")
        more.send_keys(Keys.ENTER)
    except:
        print('주소더보기 못찾음')
    
    try:
        ele = driver.find_element_by_css_selector('div.NDOdz.undefined > div:nth-child(2)').text
        address = ele.replace('지번', '').replace('복사','').replace('\n', '')
        print("c_idx : {0} / {1}".format(res[0],address))
    except:
        address = ele
        print('{0} 주소 추출실패'.format(res[0]))
        
    driver.close()
    
    try:
        sql = "UPDATE hairshop SET c_addr_jibun=%s WHERE c_idx=%s"
        val = (address, res[0])
        cursor.execute(sql, val)
        conn.commit()
    except:
        print("sql-Update Hairshop fail : {0}".format(val))
        
    
#     if response.status_code == 500:
#         url = 'https://m.place.naver.com/hairshop/{0}/home?entry=pll'.format(res[1])
#         response = requests.get(url, headers=headers)
#         time.sleep(0.5)

#     if response.status_code == 200:
#         html = response.content.decode('utf-8','replace')
#         soup = BeautifulSoup(html, 'html.parser')
#         try:
#             c_addr = soup.select_one('li._1M_Iz._1aj6- span._2yqUQ').text
#         except:
#             print("{0}의 c_addr 못찾음".format(url))
#             c_addr = -1
                    
#         c_home.append((c_phone, c_addr, c_hours, c_score, c_visitor_review, c_is_reservation, c_img_src))
#     else : 
#         print("페이지 로드 실패 : {0}".format(url))
#         print(response.status_code)


    # ----------------------------db저장---------------------------
#     print(c_home)
#     print(c_price)
#     print(c_designer)
#     print("'{0}'매장 데이터 저장완료".format(res[2]))
#     print("--------------------------------------------------------------------------------------")
    
#     try:
#         sql = "UPDATE hairshop SET c_phone=%s, c_addr=%s, c_hours=%s, c_score=%s, edit_date=%s, c_visitor_review=%s, c_is_reservation=%s, c_img_src=%s WHERE c_idx=%s"
#         val = (c_home[0][0], c_home[0][1], c_home[0][2], c_home[0][3], datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), c_home[0][4], c_home[0][5], c_home[0][6], res[0])
#         cursor.execute(sql, val)
#     except:
#         print("sql-Update Hairshop fail : {0}".format(val))
    
    
#     conn.commit()
conn.close()
print('--------------------끝-----------------')
# os.system('shutdown -s -t 120')